<a href="https://colab.research.google.com/github/RedLegacy227/Jogos/blob/main/An%C3%A1lise_MO_Home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando as Bibliotecas Básicas
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from google.colab import data_table
data_table.enable_dataframe_formatter()
# Importando a Biblioteca do KNN
from sklearn.neighbors import KNeighborsClassifier
# Importando a Biblioteca de Acurácia, Matriz de Confusão (em quantos jogos entrou em quantos foi green ou red)
from sklearn.metrics import accuracy_score, confusion_matrix
# Importando a Biblioteca SkLearn para padronizar os dados (entre 0 e 1)
from sklearn.preprocessing import StandardScaler
# Importando a Biblioteca SkLearn para dividir os dados entre treinamento e teste
from sklearn.model_selection import train_test_split


In [ ]:
# DataFrame do Campeonato Brasileiro
df = pd.read_excel("https://github.com/RedLegacy227/Jogos/blob/main/Portugal.xlsx")

In [ ]:
df = df[['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','HODD','DODD','AODD','O25','U25' ]]
df.columns = ['Date','Home','Away','Goals_H','Goals_A','Result','Odds_H','Odds_D','Odds_A','Over 25','Under 25']

In [ ]:
# Resetando o Index
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)
df = df.dropna()

In [ ]:
def Prob_Home():
    P_H = 1/df.Odds_H
    return P_H

def Prob_Draw():
    P_D = 1/df.Odds_D
    return P_D

def Prob_Away():
    P_A = 1/df.Odds_A
    return P_A

In [ ]:
df['p_H'] = Prob_Home()
df['p_D'] = Prob_Draw()
df['p_A'] = Prob_Away()

In [ ]:
def Valor_do_Gol_H():
    VG_H = df.Goals_H * df.p_A
    return VG_H

def Valor_do_Gol_A():
    VG_A = df.Goals_A * df.p_H
    return VG_A

In [ ]:
df['Valor_do_Gol_H'] = Valor_do_Gol_H()
df['Valor_do_Gol_A'] = Valor_do_Gol_A()

In [ ]:
df['Media_Valor_do_Gol_H'] = df.groupby('Home')['Valor_do_Gol_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
df['Media_Valor_do_Gol_A'] = df.groupby('Away')['Valor_do_Gol_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)

In [ ]:
def Custo_do_Gol_H():
    CG_H = df.p_H / df.Goals_H
    return CG_H

def Custo_do_Gol_A():
    CG_A = df.p_A / df.Goals_A
    return CG_A

In [ ]:
df['Custo_do_Gol_H'] = Custo_do_Gol_H()
df['Custo_do_Gol_A'] = Custo_do_Gol_A()

In [ ]:
df.replace(np.inf, 1, inplace=True)

In [ ]:
df['Media_Custo_do_Gol_H'] = df.groupby('Home')['Custo_do_Gol_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
df['Media_Custo_do_Gol_A'] = df.groupby('Away')['Custo_do_Gol_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)

In [ ]:
team = "Arouca"
df1 = df.groupby(['Home']).get_group(team)
df2 = df1[['Home','Away','Goals_H','Goals_A','Result','Media_Valor_do_Gol_H','Media_Valor_do_Gol_A','Media_Custo_do_Gol_H','Media_Custo_do_Gol_A']]
df2.tail(5)